# Usage Cohorts for a set of Users

## Cohorts Analysis 
Cohort analysis is a subset of behavioral analytics that takes the data from a given data set (e.g. an e-commerce platform, web application, or online game) and rather than looking at all users as one unit, it breaks them into related groups for analysis. These related groups, or cohorts, usually share common characteristics or experiences within a defined time-span.  

Reference Link: [Wikepedia](https://en.wikipedia.org/wiki/Cohort_analysis)


## Table of content
- Generate Sample Data
- Categorizing the users into 3 cohorts


### 1. Generate Sample Data

##### Problem Statement
- Generate random event timestamps (in a 6 month data range) for each user
- **Sessions** have multiple events from the same user within it
- Users can have multiple sessions(with events within a session) in a single day
- Sessions need to be  **time-bounded**

Session ??  
time-bounded ?? like studied for like 4 hr or such in a day and triggered event in it


In [1]:
import pandas as pd
import random, time, os
from datetime import datetime, timedelta

In [2]:
## Number of Data points to generate
# noOfOberavation = 10000
# noOfUsers, noOfEvents = 5000, 25
noOfOberavation = 10000
noOfUsers, noOfEvents = 10, 3

## Config # observation older than
n_days = 3 ## in place of 30 days

## Data will be generated for 180Days

localDataSavingName = 'dataset_Generated_10000_10_3.csv'

def strTimeProp(nodaysBefore, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """
    start = datetime.strptime(end, format) - timedelta(days=nodaysBefore)
    stime = time.mktime(time.strptime(start.strftime(format), format))
    etime = time.mktime(time.strptime(end, format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(format, time.localtime(ptime))

def randomDate(nodaysBefore, end, prop):
    return strTimeProp(nodaysBefore, end, '%Y-%m-%d %H:%M:%S', prop)

def randomEvent(totalEvent):
    ''' returns an random Event from the totall possible random event '''
    return 'event' + str(random.randint(1,totalEvent))

def randomUser(totalUser):
    return 'u' + str(random.randint(1,totalUser))
    
    
def randomDatasetGenerator():
    
    eventDict = {'userId': randomUser(noOfUsers),
                 'eventTime': randomDate(180, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), random.random()),
                'eventId': randomEvent(noOfEvents)
                }
    return eventDict
    
#randomDatasetGenerator()
if os.path.exists(localDataSavingName):
    DF = pd.read_csv(localDataSavingName)
else:
    DF = pd.DataFrame(columns= ['userId', 'eventTime', 'eventId'])
    for i in range(noOfOberavation):
        tempDF = pd.DataFrame(randomDatasetGenerator(), index=[0])
        DF = DF.append(tempDF, ignore_index=True,sort=False)

    DF.sort_values(by=['eventTime'], inplace=True)
    DF.reset_index(drop=True,  inplace=True)

    display(DF.tail())
    # DF.to_csv('dataset_Generated.csv', index=False)
    DF.to_csv(localDataSavingName, index=False)

## Categorizing users into 3 cohorts
* **Active Users (AU)**: Any user who was active at some point of time in last 30 days
* **Engaged Users (EU)**: user with >= 1 hour timespent in last 30 days (may have <40 sessions, but total timespent >= 1 hr)
* **Power Users (PU)**: user with >= 40 sessions in Embibe in last 30 days
* **Ultra Power Users (UPU)**: user with >= 100 sessions in Embibe in last 30 days


Session ?? ---  Assume --
- if even a single event has occured in a day then, will consider at least a single seesion has been present 
- in a single day if the tiime diffrence is more than some ammount then distinct session -->  Not Taken

In [3]:
## session
print('Dataset time Type', type(DF.loc[:,'eventTime'][0]))

## converting to datetime
DF['eventTime'] = [ datetime.strptime(ele, '%Y-%m-%d %H:%M:%S') for ele in DF['eventTime'] ]

print('Dataset time Type', type(DF.loc[:,'eventTime'][0]))
print(DF.shape)
DF.head()

Dataset time Type <class 'str'>
Dataset time Type <class 'pandas._libs.tslibs.timestamps.Timestamp'>
(10000, 3)


,userId,eventTime,eventId
0,u2,2018-08-18 19:41:52,event1
1,u8,2018-08-18 20:24:00,event1
2,u10,2018-08-18 20:30:06,event2
3,u2,2018-08-18 21:02:25,event3
4,u1,2018-08-18 21:45:24,event1


In [4]:
uniqueUsers = DF['userId'].unique()
print('Number of Unique users:', len(uniqueUsers))

## Transforming adding a column date
DF['date'] = [ datetime.strptime(ele.strftime('%Y-%m-%d'), '%Y-%m-%d') for ele in DF['eventTime'] ]

Number of Unique users: 10


In [5]:
## Number of Session held by user
SessionCountPerUser = DF.groupby('userId').date.count()
UserSessionsDF = pd.DataFrame(SessionCountPerUser).reset_index()
UserSessionsDF.rename(columns={'date':'noOfSessions'}, inplace=True)
print('Datset Shape:', UserSessionsDF.shape)
UserSessionsDF.head()

Datset Shape: (10, 2)


,userId,noOfSessions
0,u1,1022
1,u10,929
2,u2,970
3,u3,1029
4,u4,1027


In [6]:
## time spend by each user per day on embibe
'''
= last event on that day - first event on that day
'''
userDay = DF.groupby(['userId', 'date']).eventTime
tempDF_Max = pd.DataFrame(userDay.max()).reset_index()
tempDF_Max.rename(columns={'eventTime': 'eventTimeMax'}, inplace=True)
tempDF_Min = pd.DataFrame(userDay.min()).reset_index()
tempDF_Min.rename(columns={'eventTime': 'eventTimeMin'}, inplace=True)
tempDF = pd.merge(tempDF_Max, tempDF_Min, on=['userId', 'date'])
temp = [ tempDF['eventTimeMax'][ind] - tempDF['eventTimeMax'][ind] for ind in range(len(tempDF)) ]
tempDF['timeSpend_sec'] = [ ele.total_seconds() for ele in temp ]

print('Datset Shape:', tempDF.shape)
tempDF.head()

Datset Shape: (1800, 5)


,userId,date,eventTimeMax,eventTimeMin,timeSpend_sec
0,u1,2018-08-18,2018-08-18 21:45:24,2018-08-18 21:45:24,0.0
1,u1,2018-08-19,2018-08-19 17:05:58,2018-08-19 00:12:40,0.0
2,u1,2018-08-20,2018-08-20 22:51:41,2018-08-20 04:37:07,0.0
3,u1,2018-08-21,2018-08-21 23:52:13,2018-08-21 03:45:44,0.0
4,u1,2018-08-22,2018-08-22 19:22:52,2018-08-22 03:18:22,0.0


In [7]:
## Total timespend by user
totTimeSpendDF = pd.DataFrame(tempDF.groupby('userId').timeSpend_sec.sum()).reset_index()
print('Datset Shape:', totTimeSpendDF.shape)
display(totTimeSpendDF.head())

Datset Shape: (10, 2)


,userId,timeSpend_sec
0,u1,0.0
1,u10,0.0
2,u2,0.0
3,u3,0.0
4,u4,0.0


In [8]:

def evaluateTimeSpendAndSessions(smallDF):
    '''
    dropped  user ID
    '''
    noOfSess = smallDF.date.count()
    
    userDay = smallDF.groupby(['date']).eventTime
    tempDF_Max = pd.DataFrame(userDay.max()).reset_index()
    tempDF_Max.rename(columns={'eventTime': 'eventTimeMax'}, inplace=True)
    tempDF_Min = pd.DataFrame(userDay.min()).reset_index()
    tempDF_Min.rename(columns={'eventTime': 'eventTimeMin'}, inplace=True)
    tempDF = pd.merge(tempDF_Max, tempDF_Min, on=['date'])
    temp = [ tempDF['eventTimeMax'][ind] - tempDF['eventTimeMax'][ind] for ind in range(len(tempDF)) ]
    tempDF['timeSpend_sec'] = [ ele.total_seconds() for ele in temp ]
    totTimeSpend = tempDF.timeSpend_sec.sum()
    
    return noOfSess, totTimeSpend

DF['timespendinLast30Days'] = [ 0.0 for i in range(len(DF)) ]
DF['noOfSessioninLast30Days'] = [ 0.0 for i in range(len(DF)) ]
DF['userCohort'] = [ '-' for i in range(len(DF)) ]

for user in uniqueUsers:
    userSpecificDF = DF.loc[DF['userId']== user, :]
    for index, row in userSpecificDF.iterrows():
        # print('Processing  for user', user)
        satisf = [ ele <= row['eventTime'] - timedelta(days=n_days) for ele in userSpecificDF['date'] ]
        tempDF = userSpecificDF.loc[satisf, ['eventTime', 'eventId', 'date']]
        if len(tempDF) != 0:
            noOfSess_last30, totTimeSpendDF_last30 =evaluateTimeSpendAndSessions(tempDF)
        else:
            noOfSess_last30, totTimeSpendDF_last30 = 0, 0
        DF.loc[((DF['userId']==user)&(DF['date']==row['date'])), 'noOfSessioninLast30Days'] = noOfSess_last30
        DF.loc[((DF['userId']==user)&(DF['date']==row['date'])), 'timespendinLast30Days'] = totTimeSpendDF_last30


In [9]:
def giveCohart(row):
    ''' '''
    if row['noOfSessioninLast30Days'] >= 100:
        return 'UPU'
    elif row['noOfSessioninLast30Days'] >= 40:
        return 'PU'
    elif row['timespendinLast30Days'] > 3600.0: # an hour
        return 'EU'
    elif row['noOfSessioninLast30Days'] > 0:
        return 'AU'
    else:
        return '-'
        

li = []
for index, row in DF.iterrows():
    li.append(giveCohart(row))

DF['userCohort'] = li 

In [10]:
display(DF.tail())
DF.to_csv('result_dataset_Generated_10000_10_3.csv', index=False)

,userId,eventTime,eventId,date,timespendinLast30Days,noOfSessioninLast30Days,userCohort
9995,u9,2019-02-14 17:06:59,event3,2019-02-14,0.0,1011.0,UPU
9996,u2,2019-02-14 17:42:35,event3,2019-02-14,0.0,949.0,UPU
9997,u10,2019-02-14 18:06:35,event2,2019-02-14,0.0,908.0,UPU
9998,u2,2019-02-14 18:19:59,event1,2019-02-14,0.0,949.0,UPU
9999,u4,2019-02-14 18:34:41,event1,2019-02-14,0.0,1011.0,UPU


## Print out Day by Day

In [11]:
## print day-by-day usage cohort standings
li = []
DF.sort_values(by=['userId',  'date'], inplace=True)
userMsgNew, userMsgNext = '', ''
for index, row in DF.iterrows():
    userMsgNew = ' --- user'+ str(row['userId'].split('u')[-1]) + ' was ' + str(row['userCohort'])
    if userMsgNew != userMsgNext:
        string = 'on '+ str(row['date'].strftime('%Y-%m-%d'))+ userMsgNew
        li.append(string)
    userMsgNext = userMsgNew

li = pd.Series(li).unique()
for i in range(len(li)):
    print(li[i])

on 2018-08-18 --- user1 was -
on 2018-08-21 --- user1 was AU
on 2018-08-27 --- user1 was PU
on 2018-09-06 --- user1 was UPU
on 2018-08-18 --- user10 was -
on 2018-08-21 --- user10 was AU
on 2018-08-30 --- user10 was PU
on 2018-09-12 --- user10 was UPU
on 2018-08-18 --- user2 was -
on 2018-08-21 --- user2 was AU
on 2018-08-27 --- user2 was PU
on 2018-09-09 --- user2 was UPU
on 2018-08-19 --- user3 was -
on 2018-08-22 --- user3 was AU
on 2018-08-28 --- user3 was PU
on 2018-09-06 --- user3 was UPU
on 2018-08-18 --- user4 was -
on 2018-08-21 --- user4 was AU
on 2018-08-27 --- user4 was PU
on 2018-09-06 --- user4 was UPU
on 2018-08-19 --- user5 was -
on 2018-08-22 --- user5 was AU
on 2018-08-27 --- user5 was PU
on 2018-09-10 --- user5 was UPU
on 2018-08-18 --- user6 was -
on 2018-08-21 --- user6 was AU
on 2018-08-29 --- user6 was PU
on 2018-09-13 --- user6 was UPU
on 2018-08-19 --- user7 was -
on 2018-08-22 --- user7 was AU
on 2018-08-29 --- user7 was PU
on 2018-09-08 --- user7 was UPU
on 2

In [12]:
## print day-by-day usage cohort standings

uniqueDate = pd.Series([str(ele.strftime('%Y-%m-%d')) for ele in DF['date']]).unique()

valDF = DF.groupby(['date', 'userCohort']).userId.nunique().reset_index()
valDF.rename(columns={'userId': 'nUniqueUserId'}, inplace=True)
valDF.tail()

,date,userCohort,nUniqueUserId
187,2019-02-10,UPU,10
188,2019-02-11,UPU,10
189,2019-02-12,UPU,10
190,2019-02-13,UPU,10
191,2019-02-14,UPU,10


In [13]:
def asta(date):
    ctr = valDF.loc[valDF['date'] == date, ['userCohort', 'nUniqueUserId']]
    au = ctr.loc[ctr['userCohort'] == 'AU', 'nUniqueUserId']
    eu = ctr.loc[ctr['userCohort'] == 'EU', 'nUniqueUserId']
    pu = ctr.loc[ctr['userCohort'] =='PU', 'nUniqueUserId']
    upu = ctr.loc[ctr['userCohort'] == 'UPU', 'nUniqueUserId']
    
    au = list(au)[0] if len(au) == 1 else 0
    eu = list(eu)[0] if len(eu) == 1 else 0
    pu = list(pu)[0] if len(pu) == 1 else 0
    upu = list(upu)[0] if len(upu) == 1 else 0
    
    return au, eu, pu, upu


for date in uniqueDate:
    au, eu, pu, upu = asta(date)
    string = 'on '+ str(date)+ '\n \t\t AU: {}, EU: {}, PU: {}, UPU:{}'.format(au, eu, pu, upu)
    print(string)

on 2018-08-18
 		 AU: 0, EU: 0, PU: 0, UPU:0
on 2018-08-19
 		 AU: 0, EU: 0, PU: 0, UPU:0
on 2018-08-20
 		 AU: 0, EU: 0, PU: 0, UPU:0
on 2018-08-21
 		 AU: 7, EU: 0, PU: 0, UPU:0
on 2018-08-22
 		 AU: 10, EU: 0, PU: 0, UPU:0
on 2018-08-23
 		 AU: 10, EU: 0, PU: 0, UPU:0
on 2018-08-24
 		 AU: 10, EU: 0, PU: 0, UPU:0
on 2018-08-25
 		 AU: 10, EU: 0, PU: 0, UPU:0
on 2018-08-26
 		 AU: 10, EU: 0, PU: 0, UPU:0
on 2018-08-27
 		 AU: 6, EU: 0, PU: 4, UPU:0
on 2018-08-28
 		 AU: 5, EU: 0, PU: 5, UPU:0
on 2018-08-29
 		 AU: 2, EU: 0, PU: 8, UPU:0
on 2018-08-30
 		 AU: 0, EU: 0, PU: 10, UPU:0
on 2018-08-31
 		 AU: 0, EU: 0, PU: 10, UPU:0
on 2018-09-01
 		 AU: 0, EU: 0, PU: 10, UPU:0
on 2018-09-02
 		 AU: 0, EU: 0, PU: 10, UPU:0
on 2018-09-03
 		 AU: 0, EU: 0, PU: 10, UPU:0
on 2018-09-04
 		 AU: 0, EU: 0, PU: 10, UPU:0
on 2018-09-05
 		 AU: 0, EU: 0, PU: 10, UPU:0
on 2018-09-06
 		 AU: 0, EU: 0, PU: 7, UPU:3
on 2018-09-07
 		 AU: 0, EU: 0, PU: 7, UPU:3
on 2018-09-08
 		 AU: 0, EU: 0, PU: 4, UPU:

In [ ]:
achint@embibe.com